In [3]:
import pandas as pd
import glob

In [6]:
datafiles = glob.glob("../output/*.json")

df = pd.DataFrame()
for f in datafiles:
    _df = pd.read_json(f)
    df = df.append(_df)
df

,hash,msg,author_date,author_timezone,committer_date,committer_timezone,branches,in_main_branch,merge,parents,project_name,project_path,data_source,author_name,author_email,committer_name,committer_email,reference,fpath
0,c22ad85b5703c393f4e6ad1688f01a054dd9e230,DB: 2020-06-26\n\n2 changes to exploits/shellc...,2020-06-26T05:01:58Z,0,2020-06-26T05:01:58Z,0,[master],True,False,[e48d268df598861af9e0257a3f8a2a327990e50d],exploitdb,NaN,exploitdb_git,Offensive Security,info@exploit-db.com,Offensive Security,info@exploit-db.com,CVE-2018-11311,exploits/hardware/remote/48620.txt
1,e48d268df598861af9e0257a3f8a2a327990e50d,DB: 2020-06-25\n\n1 changes to exploits/shellc...,2020-06-25T05:01:52Z,0,2020-06-25T05:01:52Z,0,[master],True,False,[b8629afe42027db89455261b6ae2eb66efa98cf7],exploitdb,NaN,exploitdb_git,Offensive Security,info@exploit-db.com,Offensive Security,info@exploit-db.com,CVE-2020-14944,exploits/multiple/webapps/48619.txt
2,e48d268df598861af9e0257a3f8a2a327990e50d,DB: 2020-06-25\n\n1 changes to exploits/shellc...,2020-06-25T05:01:52Z,0,2020-06-25T05:01:52Z,0,[master],True,False,[b8629afe42027db89455261b6ae2eb66efa98cf7],exploitdb,NaN,exploitdb_git,Offensive Security,info@exploit-db.com,Offensive Security,info@exploit-db.com,CVE-2020-14943,exploits/multiple/webapps/48619.txt
3,b8629afe42027db89455261b6ae2eb66efa98cf7,DB: 2020-06-24\n\n4 changes to exploits/shellc...,2020-06-24T05:01:53Z,0,2020-06-24T05:01:53Z,0,[master],True,False,[09b5d3c1b686bdbae57e606afede6e29798735e8],exploitdb,NaN,exploitdb_git,Offensive Security,info@exploit-db.com,Offensive Security,info@exploit-db.com,CVE-2020-14011,exploits/windows/local/48618.txt
4,09b5d3c1b686bdbae57e606afede6e29798735e8,DB: 2020-06-23\n\n6 changes to exploits/shellc...,2020-06-23T05:02:25Z,0,2020-06-23T05:02:25Z,0,[master],True,False,[7480caf5bd53c3a34d568b2d02c4436718459813],exploitdb,NaN,exploitdb_git,Offensive Security,info@exploit-db.com,Offensive Security,info@exploit-db.com,CVE-2019-12905,exploits/multiple/webapps/48607.txt
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,e1f1ad45bcbe34ba01f6a8dde2da275abaedc63a,working exploit,2020-04-19T19:19:19Z,14400,2020-04-19T19:19:19Z,14400,[master],True,False,[58074dc6bb9d8bd29ffb8a7a93f89a19799b4935],metasploit-framework,NaN,exploitdb_git,h00die,michaeltcyr@gmail.com,h00die,michaeltcyr@gmail.com,CVE-2014-2630,modules/exploits/linux/local/hp_xglance_priv_e...
144,58074dc6bb9d8bd29ffb8a7a93f89a19799b4935,waiting on metasm question,2020-04-19T00:26:45Z,14400,2020-04-19T00:26:45Z,14400,[master],True,False,[582580d9147683d7b4f00ecac23fd21ab8c55ee7],metasploit-framework,NaN,exploitdb_git,h00die,michaeltcyr@gmail.com,h00die,michaeltcyr@gmail.com,CVE-2014-2630,modules/exploits/linux/local/hp_xglance_priv_e...
145,4f294a5deb114f1f8c21776ba22bec582c057872,"Update dependencies and finish exploit module,...",2020-03-11T23:06:36Z,18000,2020-03-11T23:06:36Z,18000,[master],True,False,[4e8eefe4ee07ec8581d0824952817378547d8966],metasploit-framework,NaN,exploitdb_git,bwatters-r7,bwatters@rapid7.com,bwatters-r7,bwatters@rapid7.com,CVE-2020-0668,modules/exploits/windows/local/cve_2020_0668_s...
146,ea64a6225a7d0867eb69bd7858a9cad915ed1076,First draft of CVE-2020-0668,2020-02-27T21:53:09Z,21600,2020-02-27T21:53:09Z,21600,[master],True,False,[0292e5c2828e1143428bddba068dbddd71c6b02e],metasploit-framework,NaN,exploitdb_git,bwatters-r7,bwatters@rapid7.com,bwatters-r7,bwatters@rapid7.com,CVE-2020-0668,modules/exploits/windows/local/cve_2020_0668_s...


In [23]:
# drop rows for hits in db/*
df = pd.DataFrame(df[~df['fpath'].str.startswith('db/')])

In [24]:
df['min_date'] = df[['author_date','committer_date']].min(axis=1)
df = df.sort_values(by=["min_date"])

In [26]:
df.groupby('reference')['min_date'].apply(lambda x: list(x))


reference
BID-1         [2013-12-03T19:44:07Z, 2013-12-03T19:44:07Z, 2...
BID-100262    [2018-04-18T00:39:59Z, 2018-04-18T00:39:59Z, 2...
BID-100467         [2018-02-09T10:14:20Z, 2018-02-09T10:14:20Z]
BID-10078     [2006-11-28T17:18:43Z, 2006-11-28T17:18:43Z, 2...
BID-10108     [2006-09-13T06:49:39Z, 2006-09-13T06:49:39Z, 2...
                                    ...                        
VU#782708                                [2015-02-10T08:36:13Z]
VU#796883                                [2013-12-03T19:44:07Z]
VU#877837                                [2019-06-04T05:01:58Z]
VU#912279          [2013-12-03T19:44:07Z, 2014-10-28T04:45:26Z]
VU#958480                                [2017-06-07T05:01:18Z]
Name: min_date, Length: 8228, dtype: object

In [17]:
df['fpath'].value_counts()[:10]

db/modules_metadata_base.json                          25475
COMMIT_MSG                                              2126
dev/db/nessus.nbe                                        264
modules/auxiliary/scanner/http/tomcat_mgr_login.rb        66
modules/exploits/multi/http/tomcat_mgr_deploy.rb          52
modules/exploits/multi/http/tomcat_mgr_upload.rb          50
lib/msf/core/db.rb                                        48
modules/exploits/windows/mssql/mssql_payload.rb           40
modules/auxiliary/dos/http/hashcollision_dos.rb           40
modules/exploits/windows/ssh/freesshd_authbypass.rb       40
Name: fpath, dtype: int64